In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt 
from PIL import Image 
import time 

%matplotlib inline 

In [2]:
import utils as utils

# Prepare Dataset 

In [3]:
BATCH_SIZE = 32 
AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically
SHUFFLE_BUFFER_SIZE = 8    # Shuffle the training data by a chunck of this many observations

In [4]:
filenames = utils.get_filenames_list('data/dataset128-easy-gaus/')
train_ds = utils.create_dataset(filenames, SHUFFLE_BUFFER_SIZE, AUTOTUNE, BATCH_SIZE)

# Model building - attempt 01 

## Architecture 

In [5]:
LATENT_DIM = 20

Model architecture from tf tutorials on MNIST dataset 

In [6]:
encoder = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(128, 128, 1)),
        tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
        tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
        tf.keras.layers.Flatten(),
        # No activation
        tf.keras.layers.Dense(LATENT_DIM + LATENT_DIM) 
        ])


In [7]:
decoder = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(LATENT_DIM,)),
        tf.keras.layers.Dense(units=8*8*32, activation=tf.nn.relu),
        tf.keras.layers.Reshape(target_shape=(8, 8, 32)),
        tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=4, padding='same',
            activation='relu'),
        tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=4, padding='same',
            activation='relu'),
        # No activation
        tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding='same'),
        ])


In [8]:
decoder.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2048)              43008     
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 64)        18496     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 128, 128, 32)      18464     
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 128, 128, 1)       289       
Total params: 80,257
Trainable params: 80,257
Non-trainable params: 0
_________________________________________________________________


Utilities for VAE 

In [9]:
@tf.function
def sample(epsilon=None):
    """ Forward pass through second half of network.  Used for generating data samples.    
        Sample epsilon using tensorflow utilities 
    """
    if epsilon is None:
        epsilon = tf.random.normal(shape=(100, LATENT_DIM))
    return decode(epsilon, apply_sigmoid=True)



def encode(X): 
    """ Forward pass of encoder network 
    
        Input is data vector, X.  
    """
    mean, logvar = tf.split(encoder(X), num_or_size_splits=2, axis=1)
    return mean, logvar 


def reparameterize(mean, logvar):
    """ Forward pass through bottleneck section of network.  Outputs z vector.  
    """
    epsilon = tf.random.normal(shape=mean.shape)
    return epsilon * tf.exp(logvar * .5) + mean




def decode(z, apply_sigmoid=False): 
    """ Forward pass of decoder network.  
    
        Input is a sampled z vector 
    """
    logits = decoder(z) 
    
    if apply_sigmoid: 
        probs = tf.sigmoid(logits)
        return probs 
    return logits 
    

# Optimizer and loss 

In [10]:
optimizer = tf.keras.optimizers.Adam(1e-4)

def log_normal_pdf(sample, mean, logvar, raxis=1):
    """ Utility for ELBO.
    """
    log2pi = tf.math.log(2. * np.pi)
    return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)


def compute_loss(x, y): 
    """ Calculates model loss 
    """
    # Forward pass through the network 
    mean, logvar = encode(x)
    z = reparameterize(mean, logvar)
    x_logit = decode(z)
    
    # Calculate reconstruction loss (binary cross entropy)
    # Reconstruction loss will evaluate the similarity between inp and out. 
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y)
    
    # Calculate regularization portion (ELBO) 
    logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
    logpz = log_normal_pdf(z, 0., 0.)
    logqz_x = log_normal_pdf(z, mean, logvar)
    return -tf.reduce_mean(logpx_z + logpz - logqz_x)
    

Training functions 

In [11]:
@tf.function
def train_step(x, y, optimizer):
    """Executes one training step and returns the loss.  

    This function computes the loss and gradients, and uses the latter to
    update the model's parameters.
    """
    
    # Use GradientTape to record a computation graph 
    with tf.GradientTape() as tape:
        loss = compute_loss(x, y)
        
    # The tape is used to compute gradients using the computation graph 
    trainable_vars = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)
    optimizer.apply_gradients(zip(gradients, trainable_vars))


## Trainin' 

In [12]:
# train_size = 60000
batch_size = 32
# test_size = 10000


In [13]:
epochs = 10
num_examples_to_generate = 16

In [14]:
for epoch in range(1, epochs + 1):
    
    # tic
    start_time = time.time()
    
    for train_x, train_y in train_ds: 
        train_step(train_x, train_y, optimizer)

    # toc 
    end_time = time.time()

#     display.clear_output(wait=False)
#     print('Epoch: {}, time elapse for current epoch: {}'
#         .format(epoch, end_time - start_time))


AttributeError: 'function' object has no attribute 'clear_output'

+ [x] training loop appears to work
+ [ ] build the random number seed thing in 